In [1]:
import time
import pyperclip
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC

In [2]:
def go_to_page(page_num):
    # Wait for the page box to become visible
    page_box = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "pagebox")))
    
    i = 1
    while True:
        try:
            # Find and click on the page link
            page = page_box.find_element(By.LINK_TEXT, str(page_num))
            page.click()
            time.sleep(1)
            break
        except:
            i += 4
            # Find and click on the page link
            page = page_box.find_element(By.LINK_TEXT, str(i))
            page.click()
            time.sleep(1)

In [3]:
driver = webdriver.Chrome()
driver.get("https://navi.cnki.net/knavi/newspapers/JFRB/detail?uniplatform=NZKPT")

search_box = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.ID, "J_searchTxt"))
)

# 在输入框中输入文字
search_box.send_keys("新冠")
time.sleep(2)

# press enter
search_box.send_keys(Keys.ENTER)
time.sleep(1)
sort_dropdown = driver.find_element(By.CLASS_NAME, "sort_select_default")

# hover over the sorting options dropdown
ActionChains(driver).move_to_element(sort_dropdown).perform()

# click and then select the date option
date_option = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//a[@name="DT"]'))
)
date_option.click()
time.sleep(1)

go_to_page(1)

In [4]:
# Find the <tbody> element within the table (assuming there is only one table)
tbody = driver.find_element(By.XPATH, "//table/tbody")

# Find all <tr> elements within the <tbody>
rows = tbody.find_elements(By.TAG_NAME, "tr")
for row in rows:
    link = row.find_element(By.XPATH, ".//td[@class='name']/a").get_attribute("href")
    # click on the link
    driver.get(link)
    html_link = driver.find_element(By.CLASS_NAME, "btn-html")
    html_link.click()
    time.sleep(1.1)
    break

# Please log in through WeChat, and then run the following code

In [5]:
# go to the new tab
driver.switch_to.window(driver.window_handles[1])
driver.close()
driver.switch_to.window(driver.window_handles[0])
driver.back()

In [6]:
# download the page
def download_page(page_num):
    go_to_page(page_num)
    
    # Find the <tbody> element within the table (assuming there is only one table)
    tbody = driver.find_element(By.XPATH, "//table/tbody")
    # Find all <tr> elements within the <tbody>
    rows = tbody.find_elements(By.TAG_NAME, "tr")

    links = []
    contents = []
    for row in rows:
        link = row.find_element(By.XPATH, ".//td[@class='name']/a").get_attribute("href")
        links.append(link)
        driver.get(link)
        time.sleep(1.2)
        html_link = driver.find_element(By.CLASS_NAME, "btn-html")
        html_link.click()
        time.sleep(1.5)
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(0.5)
        elements = driver.find_elements(By.XPATH, ".//*[contains(@class, 'p1') or contains(@class, 'anchor-tag')]")
        text = ""
        for element in elements:
            text += element.text
            text += "\n"
        contents.append(text)
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        driver.back()
        time.sleep(1)

    tbody_text = tbody.text
    tbody_text = tbody_text.split("\n")

    for i in range(len(tbody_text)):
        if i % 2 == 0:
            # split by the first space
            tbody_text[i] = tbody_text[i].split(" ", 1)
        else:
            tbody_text[i] = tbody_text[i].split(" ")
            if len(tbody_text[i]) == 3:
                tbody_text[i].append('0')

    # combine the odd and even rows to a large list
    tbody_text = [item for sublist in tbody_text for item in sublist]
    tbody_text = np.array(tbody_text).reshape(-1, 6)
    tbody_text = pd.DataFrame(tbody_text, columns=["Number", "Title", "Author", "banHao", "Date", "Download"])
    tbody_text["Links"] = links
    tbody_text["Contents"] = contents
    page_df = tbody_text.copy()
    return page_df

# 下面的代码可以根据自己需求调整

In [7]:
df21 = download_page(21)
df22 = download_page(22)
df23 = download_page(23)
# combine the three dataframes
df = pd.concat([df21, df22, df23], ignore_index=True)

In [10]:
# 根据日期筛选
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values(by="Date")
df = df[(df["Date"] >= "2020-04-01") & (df["Date"] <= "2020-06-01")]
df = df.reset_index(drop=True)
df.to_csv("data.csv", index=False, encoding="utf-8-sig")